# Do influentials drive large post diffusions?

### Our hypothesis

**Law of the Few** If a few specific set of people endorse our post, the chance of exponential success is higher


In [8]:
using Distributions, LightGraphs, ProgressMeter

using DataFrames, RCall

srand(20130810)

MersenneTwister(UInt32[0x01332bfa], Base.dSFMT.DSFMT_state(Int32[-1772545288, 1073534108, 1077066014, 1072915095, -2146195133, 1072843413, 301764553, 1073404181, 750472136, 1073628106  …  -1491411563, 1073194977, 716119449, 1072893711, 1632331784, 758890923, 1433693833, -13012230, 382, 0]), [1.36441, 1.52387, 1.89173, 1.62463, 1.45305, 1.46875, 1.79658, 1.04989, 1.38748, 1.26837  …  1.7184, 1.96956, 1.00141, 1.49897, 1.8117, 1.30385, 1.90919, 1.81893, 1.70036, 1.92811], 382)

Before we begin our exploration, it is instructive to look at the degree distribution of the underlying network on which we simulate the diffusion process. At the outset, note that we focus only on networks that have a power law distribution of node degrees in this workbook. This means that while most nodes have a small degree, there is a finite probability that some nodes have very high degree. We note that several empirical network distributions (hosted at http://konect.uni-koblenz.de/plots/degree_distribution) are networks with heavy-tailed distributions. In this context, our assumption of power law distribution of degrees is not way off-mark.

Let us begin by plotting degree distributions of a few Barabasi-Albert graphs using the builtin graph generator from LightGraphs. First, notice the correspondence between the inputs to the graph generator and the average degree of the graph 

In [9]:
N = 10^4 # Number of nodes in the network

10000

In [10]:
d1 = degree(barabasi_albert(N, 2))
d2 = degree(barabasi_albert(N, 16))

@rput d1 d2

R"""
library(ggplot2)
library(gridExtra)

theme_set(theme_bw())

p1 = qplot(d1, geom = "histogram", bins = 100) + labs(x = "Node degree", title = "Average degree = 4")
p2 = qplot(d2, geom = "histogram", bins = 100) + labs(x = "Node degree", title = "Average degree = 32")

grid.arrange(p1, p2, nrow = 2)
"""

LoadError: REvalError: Error in library(ggplot2) : there is no package called ‘ggplot2’

## Model

We model the impact of seeding to specific individuals by running diffusion simulations in two scenarios. In the first scenario, we seed the simulation by activating a randomly chosen node from the network. In the second scenario, we seed the most connected node (i.e., highest degree) in the network.    

We make the following assumptions on the decision making behavior of the nodes in a network:

 - A node in a graph is faced with a binary decision - to engage or to not engage (e.g., with new products or discussions with friends)
 - They make this decision based on a simple rule - they compute the fraction of their neighbors that have engaged, compare it with their personal threshold, and engage if the fraction of engaged neighbors exceeds the threshold. 
 - The execution of this model is random and asynchronous, i.e., each node checks the status of its neighbors in a random order and decides whether to engage or not depending on the fraction of neighbors engaged at that point
 
 > "If at least 18% of my friends bought the new iPhone, I would want to buy it too"

In [4]:
function fraction_engaged(node::Int,
                          G::LightGraphs.SimpleGraphs.SimpleGraph,
                          node_status::BitArray)

    """
    Computes the fraction of neighbors engaged within the neighborhood
    of a given node. It uses the node status to check the engagement status of
    each of the nodes neighbors
    """
    num_engaged = sum([node_status[nbr] for nbr in neighbors(G, node)])
    
    return num_engaged/length(neighbors(G, node))
end

fraction_engaged (generic function with 1 method)

In [5]:
function update_node_status(G::LightGraphs.SimpleGraphs.SimpleGraph,
                            node_status::BitArray,
                            threshold::Vector{Float64})
    """
    This function executes the random asynchronous updates of the entire network
    at each time step. In this conceptualization, each time step comprises mini
    time steps during which a randomly shuffled node list updates.    
    We store a copy of the node status before the updation. This helps to compare
    precisely what changed in the updation
    """
    old_status = copy(node_status)

    for node in shuffle(vertices(G))
        if node_status[node] == false
            if fraction_engaged(node, G, node_status) > threshold[node]
                node_status[node] = true
            end
        end
    end

    return node_status, old_status
end

update_node_status (generic function with 1 method)

In [6]:
function diffusion_simulation(n::Int,
                              z::Int,
                              threshold::Vector{Float64},
                              n_realizations::Int)
    """
    Create a new Barabasi-Albert graph at each realization. This represents a power law distribution of node degrees and is 
    supposed to be reflective of the influence distribution among the nodes in the network.
    Two kinds of seeding is conducted - seeding the most influential person in the network (e.g., one with maximum degree) 
    and a randomly chosen person in the network.
    The network then updates till no new node activations are possible at this configuration of the network 
    and the threshold. 
    The simulation is executed a large number of times and the cumulative number of activations is counted for each case.
    
    Hyper Parameters of the model
    ----------
    1. Number of nodes in the Barabasi-Albert graph (n)
    2. Average degree (z)
    3. Threshold (distribution or a specific value)
    4. Number of realizations

    Output
    -----------
    A matrix of number of activations in random and influential seeding and time steps to stability for each case at the 
    end of each realization of the simulation

    """
    output = DataFrame(z = Int[], phi = Float64[], 
                       random_engaged = Int[], random_time = Int[], 
                       influential_engaged = Int[], influential_time = Int[])
    
    k = floor(Int, z/2) # Reason out why this relationship should be true 

    @showprogress 1 "Simulating ..." for r in 1:n_realizations
        G = barabasi_albert(n, k) # represents the influence distribution
        
        # Step 1: Explore cascades triggered by random seed
        
        # Select a single random node from the network and seed it
        node_status = falses(nv(G))
        node_status[sample(vertices(G))] = true
        
        t = 1
        new_node_status, node_status = update_node_status(G, node_status, threshold)
        
        # Keep updating node status till there are more nodes to activate
        # This requires us to keep storing the node status vector from one iteration before
        # for comparison. As long as there are more nodes that can be activated, i.e., cascade has not ended, 
        # we keep updating the nodes and incrementing the time steps
        
        while !isequal(new_node_status, node_status)
            node_status = new_node_status
            new_node_status, node_status = update_node_status(G, node_status, threshold)
            t += 1
        end
        
        random_activations, random_timesteps = sum(node_status), t
        
        # Step 2: Explore cascades triggered by most connected seed 

        # Select the node with highest degree and seed it
        # This can be easily modified to rank nodes by other centrality measures
        node_status = falses(nv(G))
        seed = sortperm(degree_centrality(G), rev=true)[1]
        node_status[seed] = true
        
        t = 1
        new_node_status, node_status = update_node_status(G, node_status, threshold)
        
        # Keep updating node status till there are more nodes to activate
 
        while !isequal(new_node_status, node_status)
            node_status = new_node_status
            new_node_status, node_status = update_node_status(G, node_status, threshold)
            t += 1
        end
        
        influential_activations, influential_timesteps = sum(node_status), t
        
        push!(output, [z, threshold[1], random_activations, random_timesteps, influential_activations, influential_timesteps])
    end

    return output

end

LoadError: [91mUndefVarError: @showprogress not defined[39m

## Results 

We can run the diffusion simulations on the parameter space $(z, \phi)$ where z is the average degree of the network and $\phi$ is the threshold distribution of the nodes in the network. We vary the average degree of the nodes and create several network structures in the process. The threshold distribution can be assumed to be uniform or drawn from a specific probability distribution. 

To illustrate this process, we run the diffusion simulations on 4 pairs of $(z, \phi)$: (4, 0.1), (4, 0.4), (100, 0.1), (100, 0.4). These pairs are reflective of a factorial design with high/low values for network degree and node thresholds. we run the simulations for 100 values at each pair. Of course, we make the simplifying assumption that all nodes in the newtork have the same threshold for activation.

In [ ]:
data1 = diffusion_simulation(N, 4, fill(0.1, N), 100); # (z, phi) = (4, 0.1) Low threshold, Low degree

In [ ]:
data2 = diffusion_simulation(N, 4, fill(0.4, N), 100); # (z, phi) = (4, 0.4) Low degree, High threshold

In [ ]:
data3 = diffusion_simulation(N, 100, fill(0.1, N), 100); # (z, phi) = (100, 0.1) High degree, Low threshold

In [ ]:
data4 = diffusion_simulation(N, 100, fill(0.4, N), 100); # (z, phi) = (100, 0.4) High degree, High threshold

In [ ]:
data = vcat(data1, data2, data3, data4)
@rput data;

In [ ]:
R"""
library(dplyr)

degree_effect = data %>% group_by(z) %>% summarize_all(funs(mean))
threshold_effect = data %>% group_by(phi) %>% summarize_all(funs(mean))
elevation = data %>% transmute(elevation = influential_engaged/random_engaged) %>% 
                     summarize(mean_elevation = mean(elevation))

"""
@rget degree_effect threshold_effect elevation;

In [ ]:
elevation

In [ ]:
degree_effect

In [ ]:
threshold_effect

If we look at the mean elevation across all the simulation runs on the sample space, it looks like influentials provide a big jump in elevation compared to average individuals. This is a big +1 for the *influential hypothesis*. A direct implication of this finding is that most marketing dollars should be directed towards identifying and 'influencing' the influencers. No wonder 'influencer marketing' was the buzz word for several years!  

This is not the full picture though. Averages are severely misleading. A key idea of the Tipping Point is that these 'special' individuals are able to drive large swathes of ordinary individuals into adopting the idea because of their endorsement. Such large scale sequence of adoptions following a single adoption are called *cascades*. To be really effective, influentials should be able to drive cascades that are large not only on average, but also in scale. They have to be really, really, *really* big. The kind of big that poor ordinary individuals cannot generate, ever. These kind of super large cascades are called *global cascades*.  

A reasonable definition of a *global cascade* is one that occupies at least 10% of the network (this is based on prior literaure, but of course it is up to you to pick a cut-off). For the 10,000 node network we consider in this notebook, global cascades are those that reach 1000 nodes. Let us now see how the influencers fare on this metric. 

In [ ]:
R"""
rand_global_cascades <- data %>% filter(random_engaged > 1000) %>% count()
influential_global_cascades <- data %>% filter(influential_engaged > 1000) %>% count()

"""

@rget rand_global_cascades influential_global_cascades;


In [ ]:
rand_global_cascades

In [ ]:
influential_global_cascades